In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale

In [5]:

df = pd.read_csv('data-police-shootings/fatal-police-shootings-data.csv')
gendermask =  df['gender'].isnull()
racemask =  df['race'].isnull()
threatmask =  df['threat_level'].isnull() | (df['threat_level'] == 'other')
fleemask =  df['flee'].isnull()
armedmask = df['armed'].isnull()
agemask = df['age'].isnull()
nonthreatmask = ~(df['threat_level'].isnull()) | (df['threat_level'] == 'other')
clean_df = df[~gendermask & ~racemask & ~threatmask & ~fleemask & ~armedmask & ~agemask]
black_unarmed = clean_df[(clean_df['race'] == 'B') & (clean_df['armed'] == 'unarmed')]
white_unarmed = clean_df[(clean_df['race'] == 'W') & (clean_df['armed']== 'unarmed')]
asian_unarmed = clean_df[(clean_df['race'] == 'A') & (clean_df['armed']== 'unarmed')]
native_unarmed = clean_df[(clean_df['race'] == 'N') & (clean_df['armed']== 'unarmed')]
hispanic_unarmed = clean_df[(clean_df['race'] == 'H') & (clean_df['armed']== 'unarmed')]

black_armed = clean_df[(clean_df['race'] == 'B') & ~(clean_df['armed'] == 'unarmed')]

black_unarmed
clean_df = clean_df.drop(['longitude','latitude','is_geocoding_exact','name','id','city','state'], axis=1)
clean_df
pd.get_dummies(clean_df)
clean_df.describe()

#not armed mask 
# unarmed = ~(non_threat['armed'] == 'unarmed')
c = df.copy()[~gendermask & ~racemask & ~fleemask & ~armedmask & ~agemask]
non_threat = df[~gendermask & ~racemask & ~fleemask & ~armedmask & ~agemask]
non_threat = non_threat.drop(['longitude','latitude','is_geocoding_exact','name','id','city','state'], axis=1)
unarmed = ~(non_threat['armed'] == 'unarmed')
othermask = ~(non_threat['race'] == 'O')

non_threat_unarmed = non_threat[~non_threat['threat_level'].str.contains('attack')]
non_threat_unarmed = non_threat_unarmed[othermask]

non_threat_unarmed = non_threat_unarmed[~unarmed].reset_index(drop=True)

# non_threat.drop(['longitude','latitude','is_geocoding_exact','name','id','city','state'], axis=1)
# clean_df[~clean_df['threat_level'].str.contains('attack')].reset_index(drop=True)
c[~c['threat_level'].str.contains('attack')].reset_index(drop=True)

non_threat_unarmed.insert(0,'deaths', range(1,len(non_threat_unarmed)+1))
non_threat_unarmed['deaths1'] = (non_threat_unarmed['race'] != non_threat_unarmed['race'].shift(1)).cumsum()
non_threat_unarmed['y'] = non_threat_unarmed.groupby('race').cumcount()+1
len(non_threat_unarmed)
# non_threat_unarmed.index = non_threat_unarmed.index + 1
non_threat_unarmed.to_csv('non_threat_unarmed.csv')
non_threat_unarmed.dtypes
races = ['H','W','B','A','N','O']
    

# clean_df
# df2.reset_index(drop=True)
# df[threatmask]

FileNotFoundError: [Errno 2] No such file or directory: 'data-police-shootings/fatal-police-shootings-data.csv'

In [ ]:
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
clean_df['date'] = pd.to_datetime(clean_df['date'])
type(clean_df['date'])
type(clean_df['date'].loc[0])
clean_df.dtypes
start = '2015-01-01'
end = '2018-01-01'
cond = (clean_df['date'] > start) & (df['date'] <= end)
clean_df.loc[cond]
#clean_df['age'].describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# clean_df.corr(method='pearson')
print(clean_df.corr())
clean_df.dtypes
clean_df['manner_of_death'] = clean_df['manner_of_death'].astype('str')
clean_df['gender'] = clean_df['gender'].astype('str')
clean_df['race'] = clean_df['race'].astype('str')
clean_df['armed'] = clean_df['armed'].astype('str')
clean_df['flee'] = clean_df['flee'].astype('str')
clean_df['threat_level'] = clean_df['threat_level'].astype('str')
# clean_df['death_by']
clean_df

In [ ]:
pd.get_dummies(clean_df['race'].head(10))
X = clean_df[['race','gender','age']]
Y = clean_df['armed']

In [ ]:

non_threat_unarmed.shape
non_threat_unarmed.head()
sns.set_style('dark')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,5), sharey=True)
races = ['A','B','H','N','W',]
y = non_threat_unarmed.groupby(['race']).deaths.count()
x = pd.Series(races)

# ax[0].bar(x,y)
# ax[1].scatter(x,y)
ax[0].bar(non_threat_unarmed.race,non_threat_unarmed.deaths1)
ax[1].scatter(x,y)

u = non_threat_unarmed.drop(['date'],axis=1)


non_threat_unarmed
u = pd.get_dummies(u,drop_first=True)
u.corr()
plt.figure(figsize=(12,10))
u_x = u.drop('y',axis=1)
u_y = u.y
sns.heatmap(u_x.corr(),annot = True)

# x_train, x_test, y_train, y_test = test_train_split()
    

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

reg = LinearRegression()
x_train, x_test, y_train, y_test = train_test_split(u_x,u_y, train_size = .8, random_state = 4)
reg.fit(x_train,y_train)
reg.coef_
reg.predict(x_test)
y_train
# plt.scatter(x_train, y_train, color = "red")
# plt.plot(x_train, reg.predict(x_train), color = "green")
# plt.title("Unarmed deaths by police")
# plt.xlabel('race')
# plt.ylabel("deaths")
# plt.show()
# logisticRegr = LogisticRegression()

In [ ]:
white_deaths = len(clean_df[clean_df['race'].str.contains('W')])
black_deaths = len(clean_df[clean_df['race'].str.contains('B')])
native_deaths = len(clean_df[clean_df['race'].str.contains('N')])
asian_deaths = len(clean_df[clean_df['race'].str.contains('A')])
hispanic_deaths = len(clean_df[clean_df['race'].str.contains('H')])
total_deaths = len(clean_df['race'])
white_deaths
black_deaths
native_deaths
asian_deaths
hispanic_deaths
total_deaths
mask15 = (non_threat_unarmed['date'] > '2015-01-01' ) & (non_threat_unarmed['date'] < '2016-01-01')
mask16 = (non_threat_unarmed['date'] > '2016-01-01' ) & (non_threat_unarmed['date'] < '2017-01-01')
mask17 = (non_threat_unarmed['date'] > '2017-01-01' ) & (non_threat_unarmed['date'] < '2018-01-01')
mask18 = (non_threat_unarmed['date'] > '2018-01-01' ) & (non_threat_unarmed['date'] < '2019-01-01')
mask19 = (non_threat_unarmed['date'] > '2019-01-01' ) & (non_threat_unarmed['date'] < '2020-01-01')
mask20 = (non_threat_unarmed['date'] > '2020-01-01' ) & (non_threat_unarmed['date'] < '2021-01-01')
mask21 = (non_threat_unarmed['date'] > '2021-01-01' ) & (non_threat_unarmed['date'] < '2022-06-01')

print(len(non_threat_unarmed[mask15]))
print(len(non_threat_unarmed[mask16]))
print(len(non_threat_unarmed[mask17]))
print(len(non_threat_unarmed[mask18]))
print(len(non_threat_unarmed[mask19]))
print(len(non_threat_unarmed[mask20]))
print(len(non_threat_unarmed[mask21]))



In [ ]:
#Creating design weight

In [ ]:
designW = white_deaths/total_deaths
designB = black_deaths/total_deaths
designN = native_deaths/total_deaths
designA = asain_deaths/total_deaths
designH = hispanic_deaths/total_deaths